<a href="https://colab.research.google.com/github/egonik-unlp/random_projects/blob/master/dataset_kaggle_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de datos desde Kaggle

Primero para poder trabajar, cargamos nuestra clave de API de kaggle y activamos la libreria

In [ ]:
from google.colab import files
uploaded = files.upload()
!mkdir /root/.kaggle 
!cp kaggle.json /root/.kaggle
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


Luego descargamos el dataset y lo descomprimimos en la carpeta datasets

In [ ]:
!kaggle datasets download -d 'epa/hazardous-air-pollutants'
!unzip /content/hazardous-air-pollutants.zip -d /content/dataset

hazardous-air-pollutants.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/hazardous-air-pollutants.zip
replace /content/dataset/epa_hap_daily_summary.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Importamos las librerias que nos serán útiles para procesar los datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

Cargamos el dataset como un DataFrame de pandas

In [ ]:
# Es muy pesado de cargar, lo pongo en una celda individual para asegurarme de que no se corra dos veces
df = pd.read_csv('dataset/epa_hap_daily_summary.csv')

Renombramos columnas

In [ ]:
df.rename( columns={
    'state_code': 'state_id',
    'county_code':'county_id',
    'parameter_code':'parameter_id',
    'method_code':'method_id',
    'site_num': 'site_id',
    'date_local':'sample_date',
}, inplace = True)

In [ ]:
df['sample_id'] = df.index

0    0
1    1
2    2
3    3
4    4
Name: sample_id, dtype: int64

In [ ]:
#preparacion para generar datos unicos

# x,y = df[['state_id','county_id' ]].astype(str).iteritems()
# df['county_unique_id'] = x[1] + y[1]
x,y = df[['county_unique_id','site_id' ]].astype(str).iteritems()
df['site_unique_id'] = x[1] + y[1]

In [ ]:
df2 = df.iloc[:int(8e5),:]


In [ ]:
state = df2[[
            'state_name',
            'state_id'
]].copy()

county= df2[[
          'county_unique_id',
          'county_name',
          'state_id'
]].copy()

site = df2[[
         'address',
         'city_name',
         'cbsa_name',
         'county_unique_id'
]].copy()

sample = df2[[
           'sample_id',
           'sample_duration',
           'sample_date',
           'units_of_measure',
           'event_type',
           'observation_count',
           'arithmetic_mean',
           'site_id',
           'parameter_id',
           'method_id'
]].copy()

parameter = df2[[
              'parameter_id',
              'parameter_name'
]].copy()

method = df2[[
           'method_id',
           'method_name'
]].copy()

In [ ]:
def save_xls(list_dfs, xls_path):
    with pd.ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()
dfs = [state, county,site, sample, parameter]

save_xls(dfs, 'database.xlsx')